In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import shap
import sklearn
from dmgpred.train import get_pipeline
from dmgpred.utils.loading import load_data
from sklearn.metrics import (
    ConfusionMatrixDisplay,
)
from sklearn.model_selection import train_test_split

# print the JS visualization code to the notebook
# shap.initjs()

# sns.set_theme()
sklearn.set_config(transform_output="pandas")
np.random.seed(0)

In [ ]:
data = load_data(data_dir="../data/", processed=True)

X = data["X_train"]
y = data["y_train"]

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
pipe = get_pipeline(X_train, clf=lgb.LGBMClassifier())
pipe.fit(X_train, y_train)

In [ ]:
preprocessor = pipe.named_steps["preprocessor"]
clf = pipe.named_steps["clf"]

In [ ]:
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)
feature_names = X_test_preprocessed.columns

## Feature Importances

In [ ]:
(
    pd.Series(clf.feature_importances_, index=feature_names)
    .sort_values(ascending=True)
    .plot(kind="barh")
)

## Confusion Matrices

In [ ]:
ConfusionMatrixDisplay.from_estimator(pipe, X_test, y_test, cmap="Blues")

## SHAP Values

In [ ]:
explainer = shap.TreeExplainer(clf)
explanation = explainer(X_test_preprocessed)

In [ ]:
shap_values = explainer.shap_values(X_test_preprocessed)

In [ ]:
shap.plots.force(
    explainer.expected_value[0],
    shap_values[1, :, 1],
    link="logit",
    feature_names=feature_names,
    matplotlib=True,
)

In [ ]:
shap.plots.force(
    explainer.expected_value[0],
    shap_values[::100, :, 1],
    link="logit",
    feature_names=feature_names,
)

In [ ]:
shap.summary_plot(shap_values[:, :, 1], X_test_preprocessed)

In [ ]:
shap.plots.violin(
    shap_values[:, :, 1],
    X_test_preprocessed,
    feature_names=feature_names,
    plot_type="layered_violin",
)

In [ ]:
shap.plots.violin(shap_values[:, :, 2], feature_names=feature_names)

## Wrong Predictions

In [ ]:
y_pred = pipe.predict(X_test)

tmp = X_test.copy()
tmp["y_pred"] = y_pred
tmp["y_true"] = y_test

wrong_preds = tmp.query("y_true != y_pred").copy()
del tmp

In [ ]:
shap_values_wrong = shap_values[wrong_preds.reset_index().index]
shap_values_wrong.shape

In [ ]:
1 - wrong_preds.shape[0] / X_test.shape[0]

In [ ]:
shap.plots.force(
    explainer.expected_value[0],
    shap_values_wrong[0, :, 1],
    link="logit",
    feature_names=feature_names,
    matplotlib=True,
)